# Visualisation

This file contain tools to visualise "pt" graph and "pkl" arc file. This can be used to see the input of the neural network and the preprocessing state.

## Display pt file

In [ ]:
file_path = r"C:\Users\Cardiff\Documents\Programmation\AMGNN\data\light\processed\Thermomecha95428laser200W_at_step_00015.pt"
name = "step 15"  # Display name for polyscope

In [1]:
import torch
import polyscope as ps
import numpy as np
from pathlib import Path
from utils.config import read_config


configuration = read_config(Path("configs"))
ps.set_up_dir("z_up")
ps.init()


In [2]:
configuration

{'raw_data': 'data/light',
 'project': 'AMGNN',
 'float_type': 'float32',
 'pooling_process': 2,
 'default_powder_temperature': 25,
 'scaling_temperature': 3000,
 'scaling_power': 1000,
 'scaling_speed': 1500,
 'scaling_size': 50,
 'optimizer': 'adam',
 'learning_rate': 0.01,
 'neighbour_radius': '2e-03',
 'neighbour_k': 26,
 'input_channels': 11,
 'hidden_channels': 128,
 'out_channels': 4,
 'aggregator': 'mean',
 'distance_upper_bound': '1e-6'}

In [3]:
graph = torch.load(file_path)

In [4]:
# Y = temperature, X displacement, Y displaement, Z displament
# X = laser_speed, laser_power, layer_thickness, time_step, type, type, type, past_temp, past_x_displacement, past_y_displacement, past_z_displacement

graph.y[0], graph.x[0], graph.pos[0]

(tensor([ 1.5338e-01,  7.2329e-06,  1.8768e-07, -1.1748e-06]),
 tensor([ 6.6667e-04,  2.0000e-01,  3.0000e-01,  2.5000e-05,  0.0000e+00,
          0.0000e+00,  1.0000e+00,  1.5310e-01,  7.3669e-06,  2.0624e-07,
         -1.2093e-06]),
 tensor([0.1630, 0.1770, 0.0000]))

In [5]:
y_temperature = graph.y[:,0] * configuration["scaling_temperature"]
y_disp_x = graph.y[:,1] * configuration["scaling_size"]
y_disp_y = graph.y[:,2] * configuration["scaling_size"]
y_disp_z = graph.y[:,3] * configuration["scaling_size"]

y_displacement_vectors = np.vstack((y_disp_x, y_disp_y, y_disp_z)).T

x_laser_speed = graph.x[:,0] * configuration["scaling_speed"]
x_laser_power = graph.x[:,1] * configuration["scaling_power"]
x_layer_thickness = graph.x[:,2]
x_time_step = graph.x[:,3]
x_type = np.argmax(graph.x[:,4:7], axis=1)
x_past_temp = graph.x[:,7] * configuration["scaling_temperature"]
x_past_x_displacement = graph.x[:,8] * configuration["scaling_size"]
x_past_y_displacement = graph.x[:,9] * configuration["scaling_size"]
x_past_z_displacement = graph.x[:,10] * configuration["scaling_size"]
          

In [6]:
coordinate = graph.pos.numpy()
edge_index = graph.edge_index.numpy()

ps_net = ps.register_curve_network(name, coordinate, np.array(edge_index).T,
                                                radius=0.002)
ps_net.add_scalar_quantity("Y temperature", y_temperature, cmap="jet")
ps_net.add_scalar_quantity("Y displacement X", y_disp_x, cmap="jet")
ps_net.add_scalar_quantity("Y displacement Y", y_disp_y, cmap="jet")
ps_net.add_scalar_quantity("Y displacement Z", y_disp_z, cmap="jet")
ps_net.add_vector_quantity("Y Deformation vectors", y_displacement_vectors)


data = {
    "laser speed": x_laser_speed,
    "laser power": x_laser_power,
    "layer thickness": x_layer_thickness,
    "time step": x_time_step,
    "type": x_type,
    "past temperature": x_past_temp,
    "past displacement X" : x_past_x_displacement,
    "past displacement Y" : x_past_y_displacement,
    "past displacement Z" : x_past_z_displacement,
}

for key in data.keys():
    
    ps_net.add_scalar_quantity(f"X {key}", data[key], cmap="jet")


In [7]:
ps.show()

## Display pkl arc file

In [20]:
import pickle

In [23]:
file = r"C:\Users\Cardiff\Documents\Programmation\AMGNN\data\light\tmp_arc\Thermomecha95428laser200W_at_step_00057.pkl"

with open(file, 'rb') as pickle_file:
    arc = pickle.load(pickle_file)

In [25]:
arc.display()